<div style="border-radius:10px;
            border:#0b0265 solid;
           background-color:#7777be;
           font-size:110%;
           letter-spacing:0.5px;
            text-align: center">

<center><h1 style="padding: 25px 0px; background color:#0077be; font-weight: bold; font-family: Cursive">
Object Detection using Kalman Filter</h1></center>

</div>

The **Kalman Filter** is a recursive mathematical algorithm used for state estimation, specifically for filtering and smoothing noisy sensor measurements in a dynamic system. It provides an optimal estimate of the true state of a system based on a combination of predicted state and measurements, taking into account uncertainties and noise in both the system dynamics and measurements.

### Let's see in general what is Kalman Filter in the below example

In [1]:
#we create module name kalman filter
from kalmanfilter import KalmanFilter

In [2]:
#Intialize the kalman filter
kf=KalmanFilter()

In [3]:
import cv2
img=cv2.imread("blue_background.webp")

In [4]:
ball1_positions=[(50,100),(100,100),(150,100),(200,100),(250,100),(300,100),(350,100),(400,100),(450,100)]
ball2_positions = [(4, 300), (61, 256), (116, 214), (170, 180), (225, 148), (279, 120), (332, 97),
         (383, 80), (434, 66), (484, 55), (535, 49), (586, 49), (634, 50),
         (683, 58), (731, 69), (778, 82), (824, 101), (870, 124), (917, 148),
         (962, 169), (1006, 212), (1051, 249), (1093, 290)]
for pt in ball2_positions:
    cv2.circle(img,pt,15,(0,20,220),-1)
    
    predicted=kf.predict(pt[0],pt[1])
    #give sea blue colour for predicted
    cv2.circle(img,predicted,15,(220,220,0),4)

In [5]:
for i in range(10):
    predicted=kf.predict(predicted[0],predicted[1])
    cv2.circle(img,predicted,15,(220,220,0),4)
    
    print(predicted)

(1180, 364)
(1223, 401)
(1266, 438)
(1309, 475)
(1352, 512)
(1395, 549)
(1437, 586)
(1479, 622)
(1521, 658)
(1563, 694)


In [6]:
#This will create an image showing trajectory of the prediction
cv2.imshow("Img",img)
cv2.waitKey(0)

-1

### Let's see our actual problerm statement

Here, the object we are detecting is Pen 
I am creating a module named **PenColorDetector** where , I mention all the characteristics of the pen that the machine can identify

Then after , I am inserting the video file where, I am executing a controlled hand-off of the pen from one hand to the other. 

In [7]:
import cv2
from PenColorDetector import PenColorDetector
from kalmanfilter import KalmanFilter

cap = cv2.VideoCapture("pen_detectvedio.mp4")

# Load detector
od = PenColorDetector()

# Load Kalman filter to predict the trajectory
kf = KalmanFilter()

while True:
    ret, frame = cap.read()
    if ret is False:
        break

    pen_box = od.detect(frame)
    x, y, x2, y2 = pen_box
    cx = int((x + x2) / 2)
    cy = int((y + y2) / 2)

    predicted = kf.predict(cx, cy)
    #cv2.rectangle(frame, (x, y), (x2, y2), (255, 0, 0), 4)
    cv2.circle(frame, (cx, cy), 20, (0, 0, 255), 4)                  
    cv2.circle(frame, (predicted[0], predicted[1]), 20, (255, 0, 0), 4)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(150)
    if key == 27:
        break

In summary, the Kalman Filter is a powerful algorithm that facilitates precise prediction of the future state or outcome of an object. By effectively incorporating system dynamics, noisy measurements, and uncertainties, the Kalman Filter provides accurate estimations, enabling reliable predictions of the object's next state.

In [2]:
import cv2
from PenColorDetector import PenColorDetector
from kalmanfilter import KalmanFilter

cap = cv2.VideoCapture("pen_detectvedio.mp4")

# Load detector
od = PenColorDetector()

# Load Kalman filter to predict the trajectory
kf = KalmanFilter()

# Get the original video's frame size and FPS
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Create a VideoWriter object to save the processed frames
output_video = cv2.VideoWriter("output_video.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

while True:
    ret, frame = cap.read()
    if ret is False:
        break

    pen_box = od.detect(frame)
    x, y, x2, y2 = pen_box
    cx = int((x + x2) / 2)
    cy = int((y + y2) / 2)

    predicted = kf.predict(cx, cy)
    cv2.rectangle(frame, (x, y), (x2, y2), (255, 0, 0), 4)
    cv2.circle(frame, (cx, cy), 20, (0, 0, 255), 4)                  
    cv2.circle(frame, (predicted[0], predicted[1]), 20, (255, 0, 0), 4)

    cv2.imshow("Frame", frame)
    output_video.write(frame)  # Write the processed frame to the output video

    key = cv2.waitKey(150)
    if key == 27:
        break

# Release the resources
cap.release()
output_video.release()
cv2.destroyAllWindows()
